# 楽曲のコード進行を取得したい。

## Songleを用いたやり方  

https://songle.jp/ にはAPIが公開されている  
http://widget.songle.jp/docs/v1 を読むと、  
http://widget.songle.jp/api/v1/song/chord.json?url=www.youtube.com/watch?v=PqJNc9KVIZE  
のようにすればコード進行のjsonを取得できるようだ。

REST API なのでHTTPメソッドでアクセスすればよく、pythonからはrequestsでデータを取得可能。  
https://note.nkmk.me/python-requests-web-api/

In [4]:
import requests

In [7]:
song_url = "www.youtube.com/watch?v=LtSNzPyo0lA"
# ぼくらはみんな意味不明 ピノキオピー

r_get = requests.get("http://widget.songle.jp/api/v1/song/chord.json?url=" + song_url)

print(r_get.status_code)

200


In [8]:
r_get.headers

{'Server': 'nginx', 'Date': 'Wed, 19 Dec 2018 15:16:05 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Status': '200 OK', 'Cache-Control': 'max-age=0, private, must-revalidate', 'Access-Control-Allow-Origin': '*', 'ETag': 'W/"d78bac811763b859e90b56a77a7894c4"', 'X-Frame-Options': 'SAMEORIGIN', 'X-XSS-Protection': '1; mode=block', 'X-Content-Type-Options': 'nosniff', 'X-Runtime': '0.380415', 'X-Request-Id': '8e269ab0-b21a-461b-aad1-4c155acab185', 'Content-Encoding': 'gzip'}

In [10]:
r_get.json()['chords'] 
# キー'chords'に対応する値を取得する

[{'index': 0, 'start': 0, 'duration': 3283, 'name': 'N'},
 {'index': 1, 'start': 3283, 'duration': 2128, 'name': 'Abm7'},
 {'index': 2, 'start': 5411, 'duration': 1056, 'name': 'C#m7'},
 {'index': 3, 'start': 6467, 'duration': 1066, 'name': 'F#'},
 {'index': 4, 'start': 7533, 'duration': 2123, 'name': 'Abm7'},
 {'index': 5, 'start': 9656, 'duration': 1061, 'name': 'C#m7'},
 {'index': 6, 'start': 10717, 'duration': 9550, 'name': 'N'},
 {'index': 7, 'start': 20267, 'duration': 6377, 'name': 'Abm'},
 {'index': 8, 'start': 26644, 'duration': 2123, 'name': 'N'},
 {'index': 9, 'start': 28767, 'duration': 2122, 'name': 'Abm'},
 {'index': 10, 'start': 30889, 'duration': 1061, 'name': 'B/C#'},
 {'index': 11, 'start': 31950, 'duration': 1067, 'name': 'F#'},
 {'index': 12, 'start': 33017, 'duration': 2116, 'name': 'Abm7'},
 {'index': 13, 'start': 35133, 'duration': 1067, 'name': 'B/C#'},
 {'index': 14, 'start': 36200, 'duration': 1067, 'name': 'Bbm'},
 {'index': 15, 'start': 37267, 'duration': 21